In [ ]:
import torch
import matplotlib.pyplot as plt
import non_local_boxes
from matplotlib.colors import hsv_to_rgb
import numpy as np
import math
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

import warnings  
warnings.filterwarnings("ignore")  # to ignore the warning messages

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

# in ordert to have unblurred pictures
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [ ]:
non_local_boxes.evaluate.nb_columns = 100

>This notebook implements `Algorithm 4` of our manuscript and draws the new collapsing boxes thus obtained. In the manuscript, it gives Figure 10.

---
# I. Definitions: Line search, Draw new collapsing boxes

### Nonlocal Boxes

In [ ]:
matrix_to_tensor = non_local_boxes.utils.matrix_to_tensor
tensor_to_matrix = non_local_boxes.utils.tensor_to_matrix

# Boxes as 2x2x2x2 tensors
PR = matrix_to_tensor(non_local_boxes.utils.PR)
PRprime = matrix_to_tensor(non_local_boxes.utils.PRprime)
SR = matrix_to_tensor(non_local_boxes.utils.SR)
I = matrix_to_tensor(non_local_boxes.utils.I)
P0 = matrix_to_tensor(non_local_boxes.utils.P_0)
P1 = matrix_to_tensor(non_local_boxes.utils.P_1)
def PNL(a,b,c): return matrix_to_tensor(non_local_boxes.utils.P_NL(a,b,c))
def PL(a,b,c,d): return matrix_to_tensor(non_local_boxes.utils.P_L(a,b,c,d))

m = non_local_boxes.evaluate.nb_columns
W_BS09 = non_local_boxes.utils.W_BS09(m)

def WinCHSH(P): return float(non_local_boxes.evaluate.h_flat(P))                    # P is a 2x2x2x2 tensor
def WinCHSHprime(P): return float(non_local_boxes.evaluate.h_prime_flat(P))         # P is a 2x2x2x2 tensor
phi_flat = non_local_boxes.evaluate.phi_flat
def BoxProduct(W,P,Q): return non_local_boxes.evaluate.R_tensor(W,P,Q)[:,:,:,:,0]
def BoxProduct_matrices(W,P,Q): return tensor_to_matrix(non_local_boxes.evaluate.R(W,P,Q)[:,:,:,:,0]) # Boxes are 4x4 matrices

### Projection of a wiring $\mathsf W$ into the constraint set $\mathcal W$

In [ ]:
projected_wiring = non_local_boxes.utils.projected_wiring

### Line Search

Standard reference in Numerical Optimization: https://doi.org/10.1007/b98874

$$
\left\{
\begin{array}{l}
    \alpha^*_k = \argmax_\alpha \phi(x_k + \alpha \nabla \phi(x_k))\\
    x_{k+1}= \texttt{proj}(x_k + \alpha^*_k \nabla \phi(x_k))
\end{array}
\right.
$$

In [ ]:
def reorder_list(L, phi):
    j=0
    while j<len(L):
        if j!=0 and phi[L[j-1]]<phi[L[j]]:
            L[j-1],L[j]=L[j],L[j-1]
            j-=2
        j+=1
    return L

In [ ]:
def select_best_columns(W, P, Q, integer):
    # P,Q are 4x4 matrices
    if integer==0: return non_local_boxes.utils.random_wiring(m).detach()
    # L is the list of the "best" indexes of the columns of W
    # At the begining, we take the first indexes of W
    # We will change the list L by comparing the value at the other indexes
    # When we add a term to L, we also remove the "worst" one, and we re-order the list L
    L = [*range(integer)]
    # phi is the list of values:
    phi= phi_flat(W,P,Q).tolist()
    # We re-order the list L:
    L = reorder_list(L, phi)
    for i in range(integer, m):
        if phi[i]>phi[L[-1]]:
            L[-1]=i # We remove and replace the worst index
            L = reorder_list(L, phi)

    W_new = non_local_boxes.utils.random_wiring(m).detach()
    for k in range(integer):
        W_new[:,k] = W[:,L[k]] # we keep only the best ones

    return W_new

In [ ]:
def line_search_with_resets(P, Q, LS_iterations, K_reset, chi):
    # P,Q are 4x4 matrices
    W, external_grad = torch.zeros(32,m), torch.ones(m)
    Krange, LSrange = range(K_reset), range(LS_iterations)
    
    for j in range(0,int(1/chi)):
        # Reset some of the wirings:
        W = select_best_columns(W, P, Q, min(m, int(j*m*chi))).detach()
        W.requires_grad=True

        # At the end, we do a lot of steps:
        if j==int(1/chi)-1:  Krange=range(10*K_reset)

        # Line search:
        for _ in Krange:
            phi_flat(W, P, Q).backward(gradient=external_grad)
            gradient=W.grad
            alpha = torch.ones(m)*0.01
            Gains = phi_flat(W, P, Q)
            Gains_futur = phi_flat(W + alpha*gradient, P, Q)
            for _ in LSrange:
                mask = 0.0 + (Gains>Gains_futur)
                alpha = 0.8*mask*alpha + 1.3*(1-mask)*alpha
                Gains = torch.max(Gains, Gains_futur)
                Gains_futur = phi_flat(W + alpha*gradient, P, Q)
            W = projected_wiring(W + alpha*gradient).detach()
            W.requires_grad=True

    return W

### Draw the new collapsing boxes

To draw $\mathcal Q$, we use this article: https://arxiv.org/abs/quant-ph/0309137

In [ ]:
def Compute_Coeff(P1, P2, P3, G_value, CHSH_value, WinGame):
    A = np.array([[WinGame(P1), WinGame(P2), WinGame(P3)],
                  [WinCHSH(P1), WinCHSH(P2), WinCHSH(P3)],
                  [1, 1, 1]])
    b = np.array([G_value, CHSH_value, 1])
    return np.linalg.solve(A,b).tolist()

def E(Box, x, y):
    return Box[0,0,x,y] + Box[1,1,x,y] - Box[0,1,x,y] - Box[1,0,x,y]

def is_in_Q(G_value,CHSH_value, Box1, Box2, Box3, WinGame):
    alpha, beta, gamma = Compute_Coeff(Box1, Box2, Box3, G_value, CHSH_value, WinGame)
    Box = alpha*Box1 + beta*Box2 + gamma*Box3
        # Box is a 2x2x2x2 tensor
    coeff00 = np.arcsin(E(Box,0,0))
    coeff01 = np.arcsin(E(Box,0,1))
    coeff10 = np.arcsin(E(Box,1,0))
    coeff11 = np.arcsin(E(Box,1,1))
    return coeff00 + coeff01 + coeff10 - coeff11 <= np.pi

In [ ]:
def eta(P, x, y):
    return 2*P[0,x*y,x,y] + 2*P[1,(1+x*y)%2,x,y] - 1

def is_in_BBP23(G_value,CHSH_value, Box1, Box2, Box3, WinGame):
    alpha, beta, gamma = Compute_Coeff(Box1, Box2, Box3, G_value, CHSH_value, WinGame)
    P = alpha*Box1 + beta*Box2 + gamma*Box3
    A = (eta(P,0,0)+eta(P,0,1)+eta(P,1,0)+eta(P,1,1))**2
    B = 2*eta(P,0,0)**2+4*eta(P, 1,0)*eta(P,0,1) + 2*eta(P, 1,1)**2
    return A+B>16

In [ ]:
def is_in_BS09(P, max_power):
    if WinCHSH(P)>(3+math.sqrt(6))/6: return True
    W = non_local_boxes.utils.W_BS09(non_local_boxes.evaluate.nb_columns)
    for _ in range(max_power):
        P = BoxProduct(W, P, P)
        if WinCHSH(P)>(3+math.sqrt(6))/6: return True
    return False

In [ ]:
def is_in_here(P, max_power, LS_iterations, K_reset, chi, print_details=False):
    # cf. "Algorithm 4" in the article
    # Here we turn the 2x2x2x2 tensors into 4x4 matrices
    P = tensor_to_matrix(P)
    Pn = P
    for _ in range(max_power):
        if print_details: print("Win CHSH(Pn) = ", WinCHSH(matrix_to_tensor(Pn)))
        W = line_search_with_resets(Pn, P, LS_iterations, K_reset, chi).detach()
        L = phi_flat(W,Pn,P)
        if print_details: print("max =", float(torch.max(L)), "mean =", float(torch.mean(L)), "min =", float(torch.min(L)))
        W = select_best_columns(W, Pn, P, 1)
        L = phi_flat(W,Pn,P)
        Pn = BoxProduct_matrices(W, Pn, P).detach()
        if WinCHSH(matrix_to_tensor(Pn))>(3+math.sqrt(6))/6:   return True
    return False

In [ ]:
def draw_collapsing_boxes(P1, P1name, P2, P2name, P3, P3name, WinGame, max_power, LS_iterations, K_reset, chi, precision=0.001, precisionQ=1e-4, markersize=4, big=True, print_legend=True, print_details=False):
    # P, P1, P2, P3 are 2x2x2x2 matrices

    if non_local_boxes.evaluate.nb_columns<100:
        print("\n   WARNING: Increase the number of columns to at least 100 please (in `non_local_boxes.evaluate`).\n")

    # Draw the initial background triangle:
    x1, y1 = WinGame(P1), WinCHSH(P1)
    x2, y2 = WinGame(P2), WinCHSH(P2)
    x3, y3 = WinGame(P3), WinCHSH(P3)

    # Check if our coordinate system is good, i.e. if projected points aren't aligned:
    if (x1 - x3)*(y2-y3)==(x2-x3)*(y1-y3):
        print("\n    WARNING: Error in the coordinate system.")
        return None
    
    # Colors:
    BBLMTU_color = hsv_to_rgb([0.475, 1, 0.5])
    BS09_color = hsv_to_rgb([0.51, 1, 0.7])
    BBP23_color = hsv_to_rgb([0.575, 1, 0.6])
    here_color = hsv_to_rgb([1/14, 1, 1])
    NS_color = hsv_to_rgb([0.475, 1, 0.9])
    Quantum_color = 'pink'

    triangle = plt.Polygon(np.array([[x1, y1], [x2,y2], [x3, y3]]), color=NS_color)
    plt.gca().add_patch(triangle)

    # Draw the Quantum area:
    G_value=min(x2,x3)
    CHSH_value=(2+np.sqrt(2))/4
    while G_value<=max(x2, x3):
        if is_in_Q(G_value, CHSH_value, P1, P2, P3, WinGame):
            plt.plot([G_value, G_value], [CHSH_value, y2], color=Quantum_color)
            G_value+=precisionQ
        else:
            if CHSH_value-precisionQ>y2:  CHSH_value= CHSH_value-precisionQ
            else:
                CHSH_value=y2
                G_value+=precisionQ
    plt.plot(0.75, 0.75,'o', markersize=0.7, color=Quantum_color)

    # Draw the collapsing area [BBLMTU06]:
    BBLMTU_value = (3+math.sqrt(6))/6
    x2_BBLMTU = x1+(x2-x1)*(1-BBLMTU_value)/(1-y2)  
    x3_BBLMTU = x1 - (x1-x3)*(1-BBLMTU_value)/(1-y3)
    BBLMTU_rectangle = plt.Polygon(np.array([[x1, y1], [x2_BBLMTU,BBLMTU_value], [x3_BBLMTU, BBLMTU_value]]), color=BBLMTU_color) # use Thales theorem
    plt.gca().add_patch(BBLMTU_rectangle)

    # Draw the collapsing area [BBP23]:
    G_value = min(x2,x3)
    CHSH_value = 0.9
    while G_value+CHSH_value<=1.5:
        if is_in_BBP23(G_value, CHSH_value, P1, P2, P3, WinGame):
            plt.plot([G_value, G_value], [CHSH_value, min(BBLMTU_value,1.5-G_value)], color=BBP23_color)
            G_value+=precisionQ
            CHSH_value = 0.75*(G_value-0.5)/(0.75-0.5) + 0.9*(G_value-0.75)/(0.5-0.75) # Lagrange interpolation
        else:
            if CHSH_value+precisionQ < BBLMTU_value: CHSH_value= min(CHSH_value+precisionQ, BBLMTU_value)
            else: 
                G_value+=precisionQ
                CHSH_value = 0.75*(G_value-0.5)/(0.75-0.5) + 0.9*(G_value-0.75)/(0.5-0.75) # Lagrange interpolation
    plt.plot([x2_BBLMTU, x3_BBLMTU], [BBLMTU_value, BBLMTU_value], linewidth=1.7, color=BBP23_color)
    
    # Draw the collapsing boxes [BS09]:
    List_BS09 = []
    alpha_max = 1-(1-BBLMTU_value)/(y1-y3)
    alpha, gamma = 0., 0.
    while gamma<0.5 and alpha<alpha_max:
        P = alpha*P1 + (1-alpha-gamma)*P2 + gamma*P3
        if is_in_BS09(P, 2*max_power):
            List_BS09.append(np.around([alpha, gamma], decimals=4).tolist())
            alpha2 = alpha
            while alpha2<alpha_max:
                P = alpha2*PR + (1-alpha2-gamma)*SR + gamma*(I+PR)/2
                plt.plot(WinGame(P), WinCHSH(P), 'o', markersize=markersize, color=BS09_color)
                alpha2= round(alpha2+precision, 4)
            gamma+=precision*(3/4)
        else: alpha+=precision
            
    # Draw the collapsing boxes [this work]:
    List_here=[]
    alpha_max = 1-(1-BBLMTU_value)/(y1-y3)
    alpha, gamma = 0., 0.
    while gamma<0.5 and alpha<alpha_max:
        alpha2, gamma2 = np.around([alpha, gamma], decimals=4).tolist()
        if print_details: print(alpha2, gamma2)
        if [alpha2, gamma2] in List_BS09:   gamma+=precision*(3/4)
        else:
            P = alpha2*P1 + (1-alpha2-gamma2)*P2 + gamma2*P3
            if is_in_here(P, max_power, LS_iterations, K_reset, chi):
                List_here.append([alpha2,gamma2])
                while alpha2<alpha_max and [alpha2,gamma2] not in List_BS09:
                    P = alpha2*P1 + (1-alpha2-gamma2)*P2 + gamma2*P3
                    plt.plot(WinGame(P), WinCHSH(P), 'o', markersize=markersize, color=here_color)
                    alpha2= round(alpha2+precision, 4)
                gamma+=precision*(3/4)
            else: alpha+=precision

    # Pyplot settings:
    plt.text(x1+0.001, y1, P1name, horizontalalignment='center',weight="bold")
    plt.text(x2, y2-0.007, P2name, verticalalignment='center',weight="bold")
    plt.text(x3, y3-0.007, P3name, verticalalignment='center',weight="bold")
    if big:  fontsize=15
    else:    fontsize=12.5
    if print_legend: plt.title(f"""Collapsing Boxes ($K_{{reset}}$={K_reset}, χ={chi}, m=$10^{int(np.log10(m))}$, M={LS_iterations}, N={max_power}).""",
                                 fontsize=fontsize)
    #plt.title("Collapsing Boxes ($K_{reset}$="+str(K_reset)+", χ="+str(chi)+string+str(LS_iterations)+", N="+str(max_power)+").")
    if big:  fontsize=12
    else:    fontsize=10
    plt.xlabel("$\mathbb{P}($win at $\mathbf{CHSH}')$", fontsize=fontsize)
    plt.ylabel("$\mathbb{P}($win at $\mathbf{CHSH})$", fontsize=fontsize)
    if print_legend: 
        handles = []
        handles.append( mpatches.Patch(color=BBLMTU_color, 
            label='Collapsing $\mathcal{N\!S}$-boxes\nfrom [BBLMTU06].') )
        handles.append( mlines.Line2D([], [],color=BS09_color, marker='o',ls='', 
            label="Collapsing $\mathcal{N\!S}$-boxes\nfrom [BS09].") )
        handles.append( mpatches.Patch(color=BBP23_color, 
            label='Collapsing $\mathcal{N\!S}$-boxes\nfrom [BBP23].') )
        handles.append( mlines.Line2D([], [],color=here_color, marker='o',ls='', 
            label="Collapsing $\mathcal{N\!S}$-boxes\nusing Algorithm 4.") )
        handles.append( mpatches.Patch(color=NS_color, label='Other $\mathcal{N\!S}$-boxes.') )
        handles.append( mpatches.Patch(color=Quantum_color, label='Quantum boxes.') )
        if big:  fontsize=10.5
        else:    fontsize=9.5
        plt.legend(handles=handles, fontsize = fontsize, loc='upper right')  
    #plt.xlim(min(x1, x2, x3), max(x1, x2, x3))
    #plt.ylim(min(y1, y2, y3), max(y1, y2, y3))
    ax=plt.gca()
    ax.relim(), ax.autoscale_view()   # auto-scale axes
    plt.show()

    return List_BS09, List_here, precision

---
# II. Draw the Collapsing Area

In [ ]:
draw_collapsing_boxes(
    P1 = PR,       P1name = "PR",
    P2 = SR,       P2name = "SR",
    P3 = (PR+I)/2, P3name = "(PR+I)/2",
    WinGame=WinCHSHprime,
    max_power= 1,       # 10
    LS_iterations = 2,  # 20
    K_reset = 2,        # 10
    chi = 0.5,          # 0.1
    precision=1.4e-2,   # 1.4e-2
    precisionQ=1e-3,    # 1e-4
    markersize=2        #2
    )

In [ ]:
draw_collapsing_boxes(
    P1 = PR,       P1name = "PR",
    P2 = P0,       P2name = "$\mathbf{P}_0$",
    P3 = (PR+PNL(0,1,0))/2, P3name = "(PR+$\mathbf{P}_{NL}^{010}$)/2",
    WinGame=WinCHSHprime,
    max_power= 1,       # 10
    LS_iterations = 2,  # 20
    K_reset = 2,        # 10
    chi = 0.5,          # 0.1
    precision=1.4e-2,   # 1.4e-2
    precisionQ=1e-3,    # 1e-4
    markersize=2        #2
    )

In [ ]:
draw_collapsing_boxes(
    P1 = PR,       P1name = "PR",
    P2 = P1,       P2name = "$\mathbf{P}_1$",
    P3 = (PR+2*P1+PNL(1,1,0))/4, P3name = "(PR+$2\cdot\mathbf{P}_1$+$\mathbf{P}_{NL}^{110}$)/4",
    WinGame=WinCHSHprime,
    max_power= 1,       # 10
    LS_iterations = 2,  # 20
    K_reset = 2,        # 10
    chi = 0.5,          # 0.1
    precision=1.4e-2,   # 1.4e-2
    precisionQ=1e-3,    # 1e-4
    markersize=2        #2
    )

---
# III. Draw Collapsing Area Using the Previous Lists

Using the lists outputted in the previous algo, we can use them as input here in order to change the drawing parameters without runing again the algo.

In [ ]:
def draw_from_lists(P1, P1name, P2, P2name, P3, P3name, WinGame, List_BS09, List_here, max_power, LS_iterations, K_reset, chi, precision=0.001, precisionQ=1e-4, markersize=4, big=True, print_legend=False):
    # P, P1, P2, P3 are 2x2x2x2 matrices

    if non_local_boxes.evaluate.nb_columns!=1:
        print("\n   WARNING: Please set the number of columns to 1.")

    # Draw the initial background triangle:
    x1, y1 = WinGame(P1), WinCHSH(P1)
    x2, y2 = WinGame(P2), WinCHSH(P2)
    x3, y3 = WinGame(P3), WinCHSH(P3)

    # Check if our coordinate system is good, i.e. if projected points aren't aligned:
    if (x1 - x3)*(y2-y3)==(x2-x3)*(y1-y3):
        print("\n    WARNING: Error in the coordinate system.")
        return None
    
    # Colors:
    BBLMTU_color = hsv_to_rgb([0.475, 1, 0.5])
    BS09_color = hsv_to_rgb([0.51, 1, 0.7])
    BBP23_color = hsv_to_rgb([0.575, 1, 0.6])
    here_color = hsv_to_rgb([1/14, 1, 1])
    NS_color = hsv_to_rgb([0.475, 1, 0.9])
    Quantum_color = 'pink'

    triangle = plt.Polygon(np.array([[x1, y1], [x2,y2], [x3, y3]]), color=NS_color)
    plt.gca().add_patch(triangle)

    # Draw the Quantum area:
    G_value=min(x2,x3)
    CHSH_value=(2+np.sqrt(2))/4
    while G_value<=max(x2, x3):
        if is_in_Q(G_value, CHSH_value, P1, P2, P3, WinGame):
            plt.plot([G_value, G_value], [CHSH_value, y2], color=Quantum_color)
            G_value+=precisionQ
        else:
            if CHSH_value-precisionQ>y2:  CHSH_value= CHSH_value-precisionQ
            else:
                CHSH_value=y2
                G_value+=precisionQ
    plt.plot(0.75, 0.75,'o', markersize=0.7, color=Quantum_color)

    # Draw the collapsing area [BBLMTU06]:
    BBLMTU_value = (3+math.sqrt(6))/6
    x2_BBLMTU = x1+(x2-x1)*(1-BBLMTU_value)/(1-y2)  
    x3_BBLMTU = x1 - (x1-x3)*(1-BBLMTU_value)/(1-y3)
    BBLMTU_rectangle = plt.Polygon(np.array([[x1, y1], [x2_BBLMTU,BBLMTU_value], [x3_BBLMTU, BBLMTU_value]]), color=BBLMTU_color) # use Thales theorem
    plt.gca().add_patch(BBLMTU_rectangle)

    # Draw the collapsing area [BBP23]:
    G_value = min(x2,x3)
    CHSH_value = 0.9
    while G_value+CHSH_value<=1.5:
        if is_in_BBP23(G_value, CHSH_value, P1, P2, P3, WinGame):
            plt.plot([G_value, G_value], [CHSH_value, min(BBLMTU_value,1.5-G_value)], color=BBP23_color)
            G_value+=precisionQ
            CHSH_value = 0.75*(G_value-0.5)/(0.75-0.5) + 0.9*(G_value-0.75)/(0.5-0.75) # Lagrange interpolation
        else:
            if CHSH_value+precisionQ < BBLMTU_value: CHSH_value= min(CHSH_value+precisionQ, BBLMTU_value)
            else: 
                G_value+=precisionQ
                CHSH_value = 0.75*(G_value-0.5)/(0.75-0.5) + 0.9*(G_value-0.75)/(0.5-0.75) # Lagrange interpolation
    plt.plot([x2_BBLMTU, x3_BBLMTU], [BBLMTU_value, BBLMTU_value], linewidth=1.7, color=BBP23_color)

    # Draw the collapsing boxes [BS09]:
    alpha_max = 1-(1-BBLMTU_value)/(y1-y3)
    for [alpha, gamma] in List_BS09:
        alpha2 = alpha
        while alpha2<alpha_max:
            P = alpha2*P1 + (1-alpha2-gamma)*P2 + gamma*P3
            plt.plot(WinGame(P), WinCHSH(P), 'o', markersize=markersize, color=BS09_color)
            alpha2+=precision
            
    # Draw the collapsing boxes [this work]:
    for [alpha, gamma] in List_here:
        alpha2 = alpha
        while alpha2<alpha_max and [alpha2,gamma] not in List_BS09:
            P = alpha2*P1 + (1-alpha2-gamma)*P2 + gamma*P3
            plt.plot(WinGame(P), WinCHSH(P), 'o', markersize=markersize, color=here_color)
            alpha2= round(alpha2+precision, 4)

    # Pyplot settings:
    plt.text(x1+0.001, y1, P1name, horizontalalignment='center',weight="bold")
    plt.text(x2, y2-0.007, P2name, verticalalignment='center',weight="bold")
    plt.text(x3, y3-0.007, P3name, verticalalignment='center',weight="bold")
    if big:  fontsize=15
    else:    fontsize=12.5
    if print_legend: plt.title(f"""Collapsing Boxes ($K_{{reset}}$={K_reset}, χ={chi}, m=$10^{int(np.log10(m))}$, M={LS_iterations}, N={max_power}).""",
                                 fontsize=fontsize)
    #plt.title("Collapsing Boxes ($K_{reset}$="+str(K_reset)+", χ="+str(chi)+string+str(LS_iterations)+", N="+str(max_power)+").")
    if big:  fontsize=12
    else:    fontsize=10
    plt.xlabel("$\mathbb{P}($win at $\mathbf{CHSH}')$", fontsize=fontsize)
    plt.ylabel("$\mathbb{P}($win at $\mathbf{CHSH})$", fontsize=fontsize)
    if print_legend: 
        handles = []
        handles.append( mpatches.Patch(color=BBLMTU_color, 
            label='Collapsing $\mathcal{N\!S}$-boxes\nfrom [BBLMTU06].') )
        handles.append( mlines.Line2D([], [],color=BS09_color, marker='o',ls='', 
            label="Collapsing $\mathcal{N\!S}$-boxes\nfrom [BS09].") )
        handles.append( mpatches.Patch(color=BBP23_color, 
            label='Collapsing $\mathcal{N\!S}$-boxes\nfrom [BBP23].') )
        handles.append( mlines.Line2D([], [],color=here_color, marker='o',ls='', 
            label="Collapsing $\mathcal{N\!S}$-boxes\nusing Algorithm 4.") )
        handles.append( mpatches.Patch(color=NS_color, label='Other $\mathcal{N\!S}$-boxes.') )
        handles.append( mpatches.Patch(color=Quantum_color, label='Quantum boxes.') )
        if big:  fontsize=10.5
        else:    fontsize=9.5
        plt.legend(handles=handles, fontsize = fontsize, loc='upper right')  
    #plt.xlim(min(x1, x2, x3), max(x1, x2, x3))
    #plt.ylim(min(y1, y2, y3), max(y1, y2, y3))
    ax=plt.gca()
    ax.relim(), ax.autoscale_view()   # auto-scale axes
    plt.show()

In [ ]:
List_BS09, List_here, precision = ([[0.014, 0.0],
  [0.308, 0.0105],
  [0.406, 0.021],
  [0.476, 0.0315],
  [0.504, 0.042],
  [0.546, 0.0525],
  [0.574, 0.063],
  [0.588, 0.0735],
  [0.602, 0.084],
  [0.616, 0.0945],
  [0.616, 0.105],
  [0.63, 0.1155]],
 [[0.182, 0.0105],
  [0.238, 0.021],
  [0.294, 0.0315],
  [0.35, 0.042],
  [0.392, 0.0525],
  [0.462, 0.063],
  [0.504, 0.0735],
  [0.546, 0.084],
  [0.574, 0.0945],
  [0.602, 0.105],
  [0.63, 0.126]],
 0.014)

draw_from_lists(
    P1 = PR,       P1name = "PR",
    P2 = SR,       P2name = "SR",
    P3 = (PR+I)/2, P3name = "(PR+I)/2",
    WinGame=WinCHSHprime,
    List_BS09=List_BS09,
    List_here=List_here,
    max_power= 10,        # 10
    LS_iterations = 20,   # 20
    K_reset = 10,         # 10
    chi = 0.1,            # 0.1
    precision=precision,  # 1.4e-2
    precisionQ=1e-3,      # 1e-4
    markersize=2,         #2
    big=False,
    print_legend=True
)

In [ ]:
List_BS09, List_here, precision = ([[0.014, 0.0],
  [0.098, 0.0105],
  [0.182, 0.021],
  [0.266, 0.0315],
  [0.336, 0.042],
  [0.392, 0.0525],
  [0.448, 0.063],
  [0.49, 0.0735],
  [0.518, 0.084],
  [0.56, 0.0945],
  [0.588, 0.105],
  [0.602, 0.1155],
  [0.63, 0.126]],
 [[0.14, 0.021],
  [0.14, 0.0315],
  [0.168, 0.042],
  [0.168, 0.0525],
  [0.182, 0.063],
  [0.182, 0.0735],
  [0.196, 0.084],
  [0.224, 0.0945],
  [0.224, 0.105],
  [0.252, 0.1155],
  [0.28, 0.126],
  [0.294, 0.1365],
  [0.308, 0.147],
  [0.308, 0.1575],
  [0.308, 0.168],
  [0.322, 0.1785],
  [0.336, 0.189],
  [0.364, 0.1995],
  [0.378, 0.21],
  [0.406, 0.2205],
  [0.406, 0.231],
  [0.462, 0.2415],
  [0.476, 0.252],
  [0.476, 0.2625],
  [0.476, 0.273],
  [0.504, 0.2835],
  [0.532, 0.294],
  [0.532, 0.3045],
  [0.546, 0.315],
  [0.574, 0.3255],
  [0.588, 0.336],
  [0.602, 0.3465],
  [0.63, 0.357]],
 0.014)

draw_from_lists(
    P1 = PR,       P1name = "PR",
    P2 = P0,       P2name = "$\mathbf{P}_0$",
    P3 = (PR+PNL(0,1,0))/2, P3name = "(PR+$\mathbf{P}_{NL}^{010}$)/2",
    WinGame=WinCHSHprime,
    List_BS09=List_BS09,
    List_here=List_here,
    max_power= 10,        # 10
    LS_iterations = 20,   # 20
    K_reset = 10,         # 10
    chi = 0.1,            # 0.1
    precision=precision,  # 1.4e-2
    precisionQ=1e-3,      # 1e-4
    markersize=2,         #2
    big=False,
    print_legend=True
)

In [ ]:
List_BS09, List_here, precision =([[0.014, 0.0],
  [0.112, 0.0105],
  [0.196, 0.021],
  [0.266, 0.0315],
  [0.336, 0.042],
  [0.378, 0.0525],
  [0.42, 0.063],
  [0.434, 0.0735],
  [0.434, 0.084],
  [0.434, 0.0945],
  [0.448, 0.105],
  [0.448, 0.1155],
  [0.448, 0.126],
  [0.462, 0.1365],
  [0.462, 0.147],
  [0.462, 0.1575],
  [0.476, 0.168],
  [0.476, 0.1785],
  [0.49, 0.189],
  [0.49, 0.1995],
  [0.504, 0.21],
  [0.504, 0.2205],
  [0.518, 0.231],
  [0.518, 0.2415],
  [0.532, 0.252],
  [0.546, 0.2625],
  [0.546, 0.273],
  [0.56, 0.2835],
  [0.574, 0.294],
  [0.588, 0.3045],
  [0.602, 0.315],
  [0.616, 0.3255],
  [0.63, 0.336]],
 [[0.126, 0.021],
  [0.154, 0.0315],
  [0.154, 0.042],
  [0.168, 0.0525],
  [0.21, 0.063],
  [0.224, 0.0735],
  [0.238, 0.084],
  [0.266, 0.0945],
  [0.294, 0.105],
  [0.294, 0.1155],
  [0.336, 0.126],
  [0.35, 0.1365],
  [0.378, 0.147],
  [0.378, 0.1575],
  [0.42, 0.168],
  [0.42, 0.1785],
  [0.448, 0.189],
  [0.476, 0.1995],
  [0.476, 0.21]],
 0.014)

draw_from_lists(
    P1 = PR,       P1name = "PR",
    P2 = P1,       P2name = "$\mathbf{P}_1$",
    P3 = (PR+2*P1+PNL(1,1,0))/4, P3name = "(PR+$2\cdot\mathbf{P}_1$+$\mathbf{P}_{NL}^{110}$)/4",
    WinGame=WinCHSHprime,
    List_BS09=List_BS09,
    List_here=List_here,
    max_power= 10,        # 10
    LS_iterations = 20,   # 20
    K_reset = 10,         # 10
    chi = 0.1,            # 0.1
    precision=precision,  # 1.4e-2
    precisionQ=1e-3,      # 1e-4
    markersize=2,         #2
    big=False,
    print_legend=True
)